<img src="logo.png" alt="Logo" width="10%">

#    Supervivencia en el Titanic: Una historia contada con datos

## 1. Introducción

El hundimiento del Titanic es uno de los naufragios más famosos de la historia. El 15 de abril de 1912, durante su viaje inaugural, el Titanic se hundió tras chocar con un iceberg, resultando en la muerte de 1,502 personas de las 2,224 que viajaban a bordo.

Este análisis busca responder a la pregunta: **¿Qué factores influyeron en la supervivencia de los pasajeros?** Exploraré variables como clase social, género, edad y más para entender patrones que podrían haber determinado quién sobrevivió a esta tragedia.

A través de este análisis, no solo estaremos aplicando técnicas de ciencia de datos, sino también contando la historia humana detrás de los números, honrando la memoria de quienes estuvieron en esta tragedia.

## 2. Preparación de los Datos

Primero, carguemos las bibliotecas necesarias y exploremos el conjunto de datos:

In [52]:
# Bibliotecas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Configuraciones para visualizaciones
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 12
sns.set(font_scale=1.2)

# Para mostrar todas las columnas del DataFrame
pd.set_option('display.max_columns', None)

# Para evitar advertencias
import warnings
warnings.filterwarnings('ignore')

In [81]:
# Cargar el dataset
df = pd.read_csv('df_titanic.csv')

# Vamo a ver las primeras filas
df.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1.0,1.0,"Allen, Miss. Elisabeth Walton",female,29.0000,0.0,0.0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1.0,1.0,"Allison, Master. Hudson Trevor",male,0.9167,1.0,2.0,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1.0,0.0,"Allison, Miss. Helen Loraine",female,2.0000,1.0,2.0,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,1.0,0.0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1.0,2.0,113781,151.5500,C22 C26,S,NaN,135.0,"Montreal, PQ / Chesterville, ON"
4,1.0,0.0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1.0,2.0,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"


### 2.1 Explorando el Dataset

Este dataset tiene información sobre los pasajeros del Titanic. Ahora vamos a ver la estructura: